In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
home_dir = "../"

import pandas as pd

In [2]:
snps_with_alfa_mane_genomicToProtMapping_df = pd.read_csv(home_dir+"data/temp_popufreq_dataset/snps_with_alfa_mane_genomicToProtMapping_raw.tsv", sep="\t")
print(snps_with_alfa_mane_genomicToProtMapping_df.columns)
print(snps_with_alfa_mane_genomicToProtMapping_df.shape)
print(snps_with_alfa_mane_genomicToProtMapping_df["SNP_ID"].unique().shape[0])
print(snps_with_alfa_mane_genomicToProtMapping_df["GENE_NAME"].unique().shape[0])
print(snps_with_alfa_mane_genomicToProtMapping_df["RefSeq_prot"].unique().shape[0])
print(snps_with_alfa_mane_genomicToProtMapping_df["genomic_to_prot_variants_mapping_source"].value_counts())

/tmp/ipykernel_446979/2569974387.py:1: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  snps_with_alfa_mane_genomicToProtMapping_df = pd.read_csv(home_dir+"data/temp_popufreq_dataset/snps_with_alfa_mane_genomicToProtMapping_raw.tsv", sep="\t")


Index(['SNP_ID', 'SNP_CLASS', 'CLINICAL_SIGNIFICANCE', 'ACC', 'CHR', 'CHRPOS',
       'SPDI', 'FXN_CLASS', 'VALIDATED', 'TAX_ID', 'CREATEDATE', 'UPDATEDATE',
       'DOCSUM', 'NP_VARIANTS', 'REF_ALLELE', 'ALT_ALLELES', 'CHROM', 'POS',
       'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMN10492695',
       'SAMN10492696', 'SAMN10492697', 'SAMN10492698', 'SAMN10492699',
       'SAMN10492700', 'SAMN10492701', 'SAMN10492702', 'SAMN11605645',
       'SAMN10492703', 'SAMN10492704', 'SAMN10492705', 'GENE_NAME', 'GENE_ID',
       'RefSeq_nuc', 'RefSeq_prot', 'MANE_status', 'GRCh38_chr', 'chr_start',
       'chr_end', 'chr_strand', 'mane_protein_variants',
       'genomic_to_prot_variants_mapping_source',
       'genomic_to_mapped_prot_variants'],
      dtype='object')
(1085924, 49)
1082770
16641
16672
Source-ALT-ALLELES    1009874
uncertain               40725
ALFA                    35325
Name: genomic_to_prot_variants_mapping_source, dtype: int64


In [3]:
certain_snps_df = snps_with_alfa_mane_genomicToProtMapping_df[snps_with_alfa_mane_genomicToProtMapping_df["genomic_to_prot_variants_mapping_source"]!="uncertain"]
print(certain_snps_df.shape)
print(certain_snps_df["SNP_ID"].unique().shape[0])
print(certain_snps_df["GENE_NAME"].unique().shape[0])
print(certain_snps_df["RefSeq_prot"].unique().shape[0])

(1045199, 49)
1042137
16640
16671


In [4]:
variants = []
for i, row in enumerate(certain_snps_df.itertuples(index=False)):
    prot_variants = row.genomic_to_mapped_prot_variants.split(",")
    alfa_alt_alleles = row.ALT.split(",") # ALFA alt alleles

    wt_population, mt_populations = int(row.SAMN10492705.split(":")[0]), row.SAMN10492705.split(":")[1].split(",")
    total_population = wt_population+sum(list(map(int, mt_populations)))

    # the following condition should never satisfy
    if len(prot_variants) != len(alfa_alt_alleles):
        print(i, row)
        raise

    for j, alt_allele in enumerate(alfa_alt_alleles):
        row_dict = row._asdict()
        row_dict["alfa_alt_allele"] = alt_allele

        v = prot_variants[j]
        row_dict["prot_variant"] = v
        row_dict["prot_acc_version"]= v.split(":")[0]
        row_dict["wt_aa"] = v.split(":")[1][2:5]
        row_dict["mt_aa"] = v.split(":")[1][-3:]
        row_dict["1indexed_prot_mt_pos"] = int(v.split(":")[1][5:-3])

        mt_population = int(mt_populations[j])
        row_dict["wt_population"] = wt_population
        row_dict["mt_population"] = mt_population
        row_dict["wt_freq"] = wt_population/total_population # freq should be computed here, since we are decomposing multiple SNVs into separate independent SNVs.
        row_dict["mt_freq"] = mt_population/total_population

        variants.append(row_dict)

variants_df = pd.DataFrame(variants)
print(variants_df.shape)

(1221571, 59)


In [5]:
print(variants_df["SNP_ID"].unique().shape[0])
print(variants_df["GENE_NAME"].unique().shape[0])
print(variants_df["RefSeq_prot"].unique().shape[0])

1042137
16640
16671


In [6]:
from Bio.PDB.Polypeptide import protein_letters_3to1 # 20 amino acids: ACDEFGHIKLMNPQRSTVWY
def is_standard_protein_variant(row):
    # print(row)
    # raise
    wt_aa = row.wt_aa
    mt_aa = row.mt_aa

    if str.upper(wt_aa) not in protein_letters_3to1: return False
    if str.upper(mt_aa) not in protein_letters_3to1: return False
    return True

In [7]:
# apply filters
x = variants_df.copy()

# removing variants where mt-population count is 0
zero_mt_popu_count_removed = x[x["mt_population"].apply(lambda y: y!=0)]
print(zero_mt_popu_count_removed.shape[0], variants_df.shape[0]-zero_mt_popu_count_removed.shape[0])

# removing variants where alfa-alt-allle not in [A, C, G, T]
alfa_alt_allele_single_nucletide = zero_mt_popu_count_removed[zero_mt_popu_count_removed["alfa_alt_allele"].apply(lambda y: len(y)==1)]
print(alfa_alt_allele_single_nucletide.shape[0], zero_mt_popu_count_removed.shape[0]-alfa_alt_allele_single_nucletide.shape[0])

# removing variants where protein variants not in standard amino-acids
standard_prot_var = alfa_alt_allele_single_nucletide[alfa_alt_allele_single_nucletide.apply(is_standard_protein_variant, axis=1)]
print(standard_prot_var.shape[0], alfa_alt_allele_single_nucletide.shape[0]-standard_prot_var.shape[0])

cleaned_variants_df = standard_prot_var.copy()

1060758 160813
1060758 0
1052676 8082


In [9]:
print(cleaned_variants_df.shape)
print(cleaned_variants_df["SNP_ID"].unique().shape[0])
print(cleaned_variants_df["GENE_NAME"].unique().shape[0])
print(cleaned_variants_df["RefSeq_prot"].unique().shape[0])

(1052676, 59)
1034905
16640
16671


In [10]:
cls_labeled_variants_df = cleaned_variants_df.copy()
cls_labeled_variants_df.loc[cls_labeled_variants_df["mt_freq"]>=.01, "class"] = "Common"
cls_labeled_variants_df.loc[(cls_labeled_variants_df["mt_freq"]<.01) & (cls_labeled_variants_df["mt_freq"]>=.001), "class"] = "Rare"
cls_labeled_variants_df.loc[(cls_labeled_variants_df["mt_freq"]<.001), "class"] = "Ultra-rare"
cls_labeled_variants_df.loc[cls_labeled_variants_df["mt_population"]==1, "class"] = "Singleton"
cls_labeled_variants_df.drop_duplicates(keep="first", inplace=True, ignore_index=True)
print(cls_labeled_variants_df["class"].value_counts())
print(cls_labeled_variants_df.shape)

Singleton     590841
Ultra-rare    397925
Rare           37702
Common         26208
Name: class, dtype: int64
(1052676, 60)


In [16]:
from utils.plots_dicts import popu_freq_class_order
def print_summary(df, classes):
    print("", "#-genes", "#-proteins", "#-protein-variants", "#-unique-genomic-variants", sep="\t")
    for cls in classes:
        cls_df = df[df["class"]==cls]
        n_genes = cls_df["GENE_NAME"].unique().shape[0]
        n_prots = cls_df["prot_acc_version"].unique().shape[0]
        n_prot_variants = cls_df["prot_variant"].shape[0] # this and the following line are same
        n_genomic_variants = cls_df[["CHROM", "POS", "REF", "alfa_alt_allele"]].drop_duplicates(keep="first").shape[0] # these keywords are from ALFAs
        print(cls, n_genes, n_prots, n_prot_variants, n_genomic_variants, sep="\t")

    total_n_genes = df["GENE_NAME"].unique().shape[0]
    total_n_prots = df["prot_acc_version"].unique().shape[0]
    total_n_prot_variants = df["prot_variant"].shape[0]
    total_n_genomic_variants = df[["CHROM", "POS", "REF", "alfa_alt_allele"]].drop_duplicates(keep="first").shape[0] # these keywords are from ALFAs
    print("total", total_n_genes, total_n_prots, total_n_prot_variants, total_n_genomic_variants, sep="\t")

print_summary(cls_labeled_variants_df, popu_freq_class_order)


	#-genes	#-proteins	#-protein-variants	#-unique-genomic-variants
Singleton	16569	16600	590841	589060
Ultra-rare	16518	16549	397925	396728
Rare	12321	12339	37702	37628
Common	9614	9623	26208	26194
total	16640	16671	1052676	1049610


In [12]:
cls_labeled_variants_df.to_csv(home_dir+"data/temp_popufreq_dataset/cls_labeled_variants_raw.tsv", index=False, sep="\t")

In [13]:
cls_labeled_variants_df.columns

Index(['SNP_ID', 'SNP_CLASS', 'CLINICAL_SIGNIFICANCE', 'ACC', 'CHR', 'CHRPOS',
       'SPDI', 'FXN_CLASS', 'VALIDATED', 'TAX_ID', 'CREATEDATE', 'UPDATEDATE',
       'DOCSUM', 'NP_VARIANTS', 'REF_ALLELE', 'ALT_ALLELES', 'CHROM', 'POS',
       'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMN10492695',
       'SAMN10492696', 'SAMN10492697', 'SAMN10492698', 'SAMN10492699',
       'SAMN10492700', 'SAMN10492701', 'SAMN10492702', 'SAMN11605645',
       'SAMN10492703', 'SAMN10492704', 'SAMN10492705', 'GENE_NAME', 'GENE_ID',
       'RefSeq_nuc', 'RefSeq_prot', 'MANE_status', 'GRCh38_chr', 'chr_start',
       'chr_end', 'chr_strand', 'mane_protein_variants',
       'genomic_to_prot_variants_mapping_source',
       'genomic_to_mapped_prot_variants', 'alfa_alt_allele', 'prot_variant',
       'prot_acc_version', 'wt_aa', 'mt_aa', '1indexed_prot_mt_pos',
       'wt_population', 'mt_population', 'wt_freq', 'mt_freq', 'class'],
      dtype='object')

In [15]:
cols = ['SNP_ID', 'GENE_NAME', 'RefSeq_prot', 'RefSeq_nuc', 'MANE_status', 'ACC', 'CHR', 
        'CHRPOS', 'CHROM', 'POS', 'REF', 'ALT', 'alfa_alt_allele', 'REF_ALLELE', 'ALT_ALLELES',
        'prot_variant', 'prot_acc_version', '1indexed_prot_mt_pos', 'wt_aa', 'mt_aa',
        'wt_population', 'mt_population', 'wt_freq', 'mt_freq', 'class']

cls_labeled_variants_df[cols]

,SNP_ID,GENE_NAME,RefSeq_prot,RefSeq_nuc,MANE_status,ACC,CHR,CHRPOS,CHROM,POS,...,prot_variant,prot_acc_version,1indexed_prot_mt_pos,wt_aa,mt_aa,wt_population,mt_population,wt_freq,mt_freq,class
0,rs2240197,MRPL23,NP_066957.3,NM_021134.4,MANE Select,NC_000011.10,11,11:1950975,NC_000011.10,1950975,...,NP_066957.3:p.Gly32Ser,NP_066957.3,32,Gly,Ser,143162,4036,0.972581,0.027419,Common
1,rs34134444,MRPL23,NP_066957.3,NM_021134.4,MANE Select,NC_000011.10,11,11:1950913,NC_000011.10,1950913,...,NP_066957.3:p.Arg11Gln,NP_066957.3,11,Arg,Gln,173910,5809,0.967677,0.032323,Common
2,rs1557425255,HMGB3,NP_005333.2,NM_005342.4,MANE Select,NC_000023.11,X,X:150987186,NC_000023.11,150987186,...,NP_005333.2:p.Gly117Ser,NP_005333.2,117,Gly,Ser,15150,1,0.999934,0.000066,Singleton
3,rs1557418082,MTMR1,NP_001293073.1,NM_001306144.3,MANE Select,NC_000023.11,X,X:150762623,NC_000023.11,150762623,...,NP_001293073.1:p.Arg639His,NP_001293073.1,639,Arg,His,10680,2,0.999813,0.000187,Ultra-rare
4,rs1557417726,MTMR1,NP_001293073.1,NM_001306144.3,MANE Select,NC_000023.11,X,X:150755690,NC_000023.11,150755690,...,NP_001293073.1:p.Asp561Gly,NP_001293073.1,561,Asp,Gly,4470,1,0.999776,0.000224,Singleton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1052671,rs5764,TBXAS1,NP_001052.3,NM_001061.7,MANE Select,NC_000007.14,7,7:140017687,NC_000007.14,140017687,...,NP_001052.3:p.Arg461Gly,NP_001052.3,461,Arg,Gly,39226,9,0.999745,0.000229,Ultra-rare
1052672,rs5764,TBXAS1,NP_001052.3,NM_001061.7,MANE Select,NC_000007.14,7,7:140017687,NC_000007.14,140017687,...,NP_001052.3:p.Arg461Trp,NP_001052.3,461,Arg,Trp,39226,1,0.999745,0.000025,Singleton
1052673,rs5279,PTGS2,NP_000954.1,NM_000963.4,MANE Select,NC_000001.11,1,1:186675946,NC_000001.11,186675946,...,NP_000954.1:p.His403Gln,NP_000954.1,403,His,Gln,59604,313,0.994760,0.005224,Rare
1052674,rs5279,PTGS2,NP_000954.1,NM_000963.4,MANE Select,NC_000001.11,1,1:186675946,NC_000001.11,186675946,...,NP_000954.1:p.His403Gln,NP_000954.1,403,His,Gln,59604,1,0.994760,0.000017,Singleton


In [17]:
cols_renaming_dict = {'SNP_ID':'snp_id', 'GENE_NAME':'gene_name', 'RefSeq_prot':'mane_refseq_prot', 'RefSeq_nuc':'mane_refseq_nuc', 'MANE_status':"mane_status", 
                      'ACC':"chrom_acc_version", 'CHR':"chrom_num", 'REF_ALLELE':"source_ref_allele", 'ALT_ALLELES':"source_alt_alleles",
                      'POS':"alfa_chrom_pos", 'REF':"alfa_ref_allele", 'ALT':"alfa_alt_alleles"}

selected_cols = ['snp_id', 'gene_name', 'mane_refseq_prot', 'mane_refseq_nuc', "mane_status", 
                 "chrom_acc_version", "chrom_num", "source_ref_allele", "source_alt_alleles",
                 "alfa_chrom_pos", "alfa_ref_allele", "alfa_alt_allele", "alfa_alt_alleles",
                 'prot_variant', 'prot_acc_version', '1indexed_prot_mt_pos', 'wt_aa', 'mt_aa',
                 'wt_population', 'mt_population', 'wt_freq', 'mt_freq', 'class']

cls_labeled_variants_short_df = cls_labeled_variants_df.rename(columns=cols_renaming_dict)
cls_labeled_variants_short_df = cls_labeled_variants_short_df[selected_cols]
cls_labeled_variants_short_df

,snp_id,gene_name,mane_refseq_prot,mane_refseq_nuc,mane_status,chrom_acc_version,chrom_num,source_ref_allele,source_alt_alleles,alfa_chrom_pos,...,prot_variant,prot_acc_version,1indexed_prot_mt_pos,wt_aa,mt_aa,wt_population,mt_population,wt_freq,mt_freq,class
0,rs2240197,MRPL23,NP_066957.3,NM_021134.4,MANE Select,NC_000011.10,11,G,"A,C",1950975,...,NP_066957.3:p.Gly32Ser,NP_066957.3,32,Gly,Ser,143162,4036,0.972581,0.027419,Common
1,rs34134444,MRPL23,NP_066957.3,NM_021134.4,MANE Select,NC_000011.10,11,G,A,1950913,...,NP_066957.3:p.Arg11Gln,NP_066957.3,11,Arg,Gln,173910,5809,0.967677,0.032323,Common
2,rs1557425255,HMGB3,NP_005333.2,NM_005342.4,MANE Select,NC_000023.11,X,G,A,150987186,...,NP_005333.2:p.Gly117Ser,NP_005333.2,117,Gly,Ser,15150,1,0.999934,0.000066,Singleton
3,rs1557418082,MTMR1,NP_001293073.1,NM_001306144.3,MANE Select,NC_000023.11,X,G,A,150762623,...,NP_001293073.1:p.Arg639His,NP_001293073.1,639,Arg,His,10680,2,0.999813,0.000187,Ultra-rare
4,rs1557417726,MTMR1,NP_001293073.1,NM_001306144.3,MANE Select,NC_000023.11,X,A,G,150755690,...,NP_001293073.1:p.Asp561Gly,NP_001293073.1,561,Asp,Gly,4470,1,0.999776,0.000224,Singleton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1052671,rs5764,TBXAS1,NP_001052.3,NM_001061.7,MANE Select,NC_000007.14,7,C,"A,G,T",140017687,...,NP_001052.3:p.Arg461Gly,NP_001052.3,461,Arg,Gly,39226,9,0.999745,0.000229,Ultra-rare
1052672,rs5764,TBXAS1,NP_001052.3,NM_001061.7,MANE Select,NC_000007.14,7,C,"A,G,T",140017687,...,NP_001052.3:p.Arg461Trp,NP_001052.3,461,Arg,Trp,39226,1,0.999745,0.000025,Singleton
1052673,rs5279,PTGS2,NP_000954.1,NM_000963.4,MANE Select,NC_000001.11,1,A,"C,G,T",186675946,...,NP_000954.1:p.His403Gln,NP_000954.1,403,His,Gln,59604,313,0.994760,0.005224,Rare
1052674,rs5279,PTGS2,NP_000954.1,NM_000963.4,MANE Select,NC_000001.11,1,A,"C,G,T",186675946,...,NP_000954.1:p.His403Gln,NP_000954.1,403,His,Gln,59604,1,0.994760,0.000017,Singleton


In [3]:
from Bio.PDB.Polypeptide import protein_letters_3to1 # 20 amino acids: ACDEFGHIKLMNPQRSTVWY

# cls_labeled_variants_short_df = pd.read_csv(home_dir+"data/temp_popufreq_dataset/cls_labeled_variants_short.tsv", sep="\t")
cls_labeled_variants_short_df["wt_aa_1letter"] = cls_labeled_variants_short_df["wt_aa"].apply(lambda x: protein_letters_3to1[str.upper(x)])
cls_labeled_variants_short_df["mt_aa_1letter"] = cls_labeled_variants_short_df["mt_aa"].apply(lambda x: protein_letters_3to1[str.upper(x)])
cls_labeled_variants_short_df

,snp_id,gene_name,mane_refseq_prot,mane_refseq_nuc,mane_status,chrom_acc_version,chrom_num,source_ref_allele,source_alt_alleles,alfa_chrom_pos,...,1indexed_prot_mt_pos,wt_aa,mt_aa,wt_population,mt_population,wt_freq,mt_freq,class,wt_aa_1letter,mt_aa_1letter
0,rs2240197,MRPL23,NP_066957.3,NM_021134.4,MANE Select,NC_000011.10,11,G,"A,C",1950975,...,32,Gly,Ser,143162,4036,0.972581,0.027419,Common,G,S
1,rs34134444,MRPL23,NP_066957.3,NM_021134.4,MANE Select,NC_000011.10,11,G,A,1950913,...,11,Arg,Gln,173910,5809,0.967677,0.032323,Common,R,Q
2,rs1557425255,HMGB3,NP_005333.2,NM_005342.4,MANE Select,NC_000023.11,X,G,A,150987186,...,117,Gly,Ser,15150,1,0.999934,0.000066,Singleton,G,S
3,rs1557418082,MTMR1,NP_001293073.1,NM_001306144.3,MANE Select,NC_000023.11,X,G,A,150762623,...,639,Arg,His,10680,2,0.999813,0.000187,Ultra-rare,R,H
4,rs1557417726,MTMR1,NP_001293073.1,NM_001306144.3,MANE Select,NC_000023.11,X,A,G,150755690,...,561,Asp,Gly,4470,1,0.999776,0.000224,Singleton,D,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1052671,rs5764,TBXAS1,NP_001052.3,NM_001061.7,MANE Select,NC_000007.14,7,C,"A,G,T",140017687,...,461,Arg,Gly,39226,9,0.999745,0.000229,Ultra-rare,R,G
1052672,rs5764,TBXAS1,NP_001052.3,NM_001061.7,MANE Select,NC_000007.14,7,C,"A,G,T",140017687,...,461,Arg,Trp,39226,1,0.999745,0.000025,Singleton,R,W
1052673,rs5279,PTGS2,NP_000954.1,NM_000963.4,MANE Select,NC_000001.11,1,A,"C,G,T",186675946,...,403,His,Gln,59604,313,0.994760,0.005224,Rare,H,Q
1052674,rs5279,PTGS2,NP_000954.1,NM_000963.4,MANE Select,NC_000001.11,1,A,"C,G,T",186675946,...,403,His,Gln,59604,1,0.994760,0.000017,Singleton,H,Q


In [6]:
cols = ['snp_id', 'gene_name', 'mane_refseq_prot', 'mane_refseq_nuc',
       'mane_status', 'chrom_acc_version', 'chrom_num', 'source_ref_allele',
       'source_alt_alleles', 'alfa_chrom_pos', 'alfa_ref_allele',
       'alfa_alt_allele', 'alfa_alt_alleles', 'prot_variant',
       'prot_acc_version', '1indexed_prot_mt_pos', 'wt_aa', 'mt_aa', 'wt_aa_1letter', 'mt_aa_1letter',
       'wt_population', 'mt_population', 'wt_freq', 'mt_freq', 'class']
cls_labeled_variants_short_df = cls_labeled_variants_short_df[cols]
cls_labeled_variants_short_df

,snp_id,gene_name,mane_refseq_prot,mane_refseq_nuc,mane_status,chrom_acc_version,chrom_num,source_ref_allele,source_alt_alleles,alfa_chrom_pos,...,1indexed_prot_mt_pos,wt_aa,mt_aa,wt_aa_1letter,mt_aa_1letter,wt_population,mt_population,wt_freq,mt_freq,class
0,rs2240197,MRPL23,NP_066957.3,NM_021134.4,MANE Select,NC_000011.10,11,G,"A,C",1950975,...,32,Gly,Ser,G,S,143162,4036,0.972581,0.027419,Common
1,rs34134444,MRPL23,NP_066957.3,NM_021134.4,MANE Select,NC_000011.10,11,G,A,1950913,...,11,Arg,Gln,R,Q,173910,5809,0.967677,0.032323,Common
2,rs1557425255,HMGB3,NP_005333.2,NM_005342.4,MANE Select,NC_000023.11,X,G,A,150987186,...,117,Gly,Ser,G,S,15150,1,0.999934,0.000066,Singleton
3,rs1557418082,MTMR1,NP_001293073.1,NM_001306144.3,MANE Select,NC_000023.11,X,G,A,150762623,...,639,Arg,His,R,H,10680,2,0.999813,0.000187,Ultra-rare
4,rs1557417726,MTMR1,NP_001293073.1,NM_001306144.3,MANE Select,NC_000023.11,X,A,G,150755690,...,561,Asp,Gly,D,G,4470,1,0.999776,0.000224,Singleton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1052671,rs5764,TBXAS1,NP_001052.3,NM_001061.7,MANE Select,NC_000007.14,7,C,"A,G,T",140017687,...,461,Arg,Gly,R,G,39226,9,0.999745,0.000229,Ultra-rare
1052672,rs5764,TBXAS1,NP_001052.3,NM_001061.7,MANE Select,NC_000007.14,7,C,"A,G,T",140017687,...,461,Arg,Trp,R,W,39226,1,0.999745,0.000025,Singleton
1052673,rs5279,PTGS2,NP_000954.1,NM_000963.4,MANE Select,NC_000001.11,1,A,"C,G,T",186675946,...,403,His,Gln,H,Q,59604,313,0.994760,0.005224,Rare
1052674,rs5279,PTGS2,NP_000954.1,NM_000963.4,MANE Select,NC_000001.11,1,A,"C,G,T",186675946,...,403,His,Gln,H,Q,59604,1,0.994760,0.000017,Singleton


In [7]:
cls_labeled_variants_short_df.to_csv(home_dir+"data/temp_popufreq_dataset/cls_labeled_variants_short.tsv", index=False, sep="\t")